In [ ]:
import argparse
import math
import os
import re
import unicodedata
from datetime import datetime, timedelta

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pytz
import seaborn as sns
from openpyxl import load_workbook

In [ ]:
file_path = os.path.expanduser('~/Downloads/report1728003166411.csv')

try:
    report_df = pd.read_csv(file_path, encoding='Latin-1')
except FileNotFoundError:
    print(f"File not found: {file_path}")
except pd.errors.ParserError:
    print(f"Error parsing the file: {file_path}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

In [ ]:
# Column renaming mapping
column_renaming_mapping = {
    'case_owner': 'owner',
    'case_owner_alias': 'owner_alias',
    'case_owner_role': 'owner_role',
    'owner_is_queue': 'is_queue',
    'case_origin': 'origin',
    'case_number': 'case_number',
    'case_reason': 'reason',
    'status': 'status',
    'open': 'is_open',
    'subject': 'subject',
    'description': 'description',
    'account_name': 'account',
    'account_record_type': 'record_type',
    'account_tier_name': 'tier',
    'parent_account': 'parent_account',
    'region': 'region',
    'datetime_opened': 'created_at',
    'time_to_in_progress_hours': 'response_hrs',
    'age_hours': 'age_hrs'
}

# Tier mapping
tier_mapping = {
    'Diamond': 'Diamond',
    'Platinum': 'Platinum',
    'Gold': 'Gold',
    'Silver': 'Silver',
    'Bronze': 'Bronze',
    'Not Tiered': 'Not Tiered',
    'Unsupported': 'Not Tiered'
}

# Region mapping
region_mapping = {
    'ASIA PACIFIC': 'APAC',
    'EMEA': 'EMEA',
    'NORTH AMERICA': 'AMER',
    'LATAM': 'AMER'
}

# Allowed regions
allowed_regions = ['APAC', 'EMEA']

# Data types for columns
column_data_types = {
    'owner': 'string',
    'owner_alias': 'string',
    'owner_role': 'string',
    'is_queue': 'bool',
    'origin': 'string',
    'case_number': 'string',
    'reason': 'string',
    'status': 'string',
    'is_open': 'bool',
    'subject': 'string',
    'description': 'string',
    'account': 'string',
    'record_type': 'string',
    'tier': 'string',
    'parent_account': 'string',
    'region': 'string',
    'created_at': 'datetime64[ns, America/Mexico_City]',
    'response_hrs': 'float',
    'age_hrs': 'float'
}

# Owner team mapping
owner_team_mapping = {
    'Team USA': ['apersaud', 'asachdev', 'asangkar', 'biron', 'gdelucen', 'ghudson', 'hpatel2',
                 'jaguirre', 'jcescoba', 'jguzm', 'kbowdenf', 'kcooper2', 'kla', 'kquan', 'lbyrom',
                 'mdardign', 'mwalton2', 'pfullmer', 'scastell', 'yjkim'],
    'Team Enroute': ['aegorova', 'aferrara', 'agarduno', 'amoreno', 'apena', 'aperez', 'arodrigu',
                     'bsussela', 'bvazquez', 'cbarbosa', 'crevilla', 'dsalas', 'eguzman', 'esolis',
                     'fdelapen', 'gdiaz', 'gmalagon', 'janovero', 'jgranado', 'jromopad', 'kalvarez',
                     'lugalde', 'mdeleon', 'mmendez', 'molais', 'msilva', 'nroldan', 'pmadero',
                     'rbravope', 'reliasfe', 'rflores', 'rgarcia', 'rvazquez', 'samaradh', 'scano',
                     'sgonzale', 'vperez', 'yegraf'],
    'Team APAC': ['abentiga', 'jbahn', 'jng', 'kpak', 'manan', 'skapoor', 'tvalerio', 'vtiama'],
    'Team EMEA': ['abentahi', 'amartos', 'bperraut', 'cadams', 'cmackenz', 'crisc', 'cyu', 'ebine',
                  'fmuehlba', 'fmulas', 'gsmit', 'hmasih', 'jcrow', 'jvoigtsb', 'kforson', 'mbendada',
                  'mbespalo', 'mcowley', 'msalami', 'mtipa', 'nicwong', 'pball', 'pbenoit', 'pprin',
                  'rde l', 'rfurlan', 'rwajahat', 'steo', 'tbillimo', 'vbuschwa'],
    'Team PUNE': ['akate', 'anaiknaw', 'anair', 'asadar', 'asharma2', 'athombre', 'awankhad', 'braghuva',
                  'dtare', 'ikahmed', 'kmakode', 'mthakur', 'nkaul', 'pdeshmuk', 'rjadhawa', 'sdutta',
                  'skeskar', 'sranjane', 'vbhedasg'],
    'IAS Bot': ['jinte'],
    'Queue': ['APAC Collections', 'Austrailia/New Zealand - Tier 1', 'Southeast Asia - Tier 1',
              'TAM APAC', 'CS US Region Support', 'TAM USA', 'USA - Tier 1', 'USA Buy Side',
              'Closed Cases Queue', 'Central Europe - Tier 1', 'EMEA Buy Side', 'EMEA Managers',
              'EMEA Publisher/Platform', 'France & Belgium - Tier 1', 'Italy - Tier 1',
              'Northern Europe - Tier 1', 'Spain & Portugal - Tier 1', 'TAM EMEA', 'India - Tier 1',
              'Theorem CS Queue', 'APAC Collections', 'Platform TAM queue',
              'Product Specialists - Walled Gardens'],
    'Non-CTAM': [],
    'CSO': ['Customer Success Operations', 'rronghan', 'ddevkate'],
    'PTAM': ['dwhite', 'mbaudist', 'arosasdi']
}

# Timezone settings
original_timezone = 'America/Mexico_City'
timezone_mapping = {
    'APAC': 'Asia/Tokyo',
    'EMEA': 'Europe/London',
    'AMER': 'America/New_York'
}

# Business hours settings
business_hours_per_day = 9  # 8 AM to 5 PM
business_days_per_week = 5
business_start_hour = 9
business_end_hour = 17

# Launch specific cases
launch_specific_cases = [
    'Unmapped Publisher', 'Tagging', 'Tag Wrapping Request', 
    'Innovid Error', 'New API Tag Wrapping Pending Request'
]

In [ ]:
def clean_column_name(column_name):
    """
    Clean and normalize a column name by:
    - Removing accents and diacritics.
    - Removing non-alphanumeric characters.
    - Replacing whitespace with underscores.
    - Converting to lowercase.
    """
    # Remove accents and diacritics
    normalized_name = ''.join(
        (char for char in unicodedata.normalize('NFD', column_name) if unicodedata.category(char) != 'Mn')
    )
    # Remove non-alphanumeric characters
    alphanumeric_name = re.sub(r'[^\w\s]', '', normalized_name)
    # Replace whitespace with underscores and convert to lowercase
    cleaned_name = re.sub(r'\s+', '_', alphanumeric_name.strip().lower())
    return cleaned_name

report_df.columns = [clean_column_name(column) for column in report_df.columns]